In [1]:
import pandas as pd
import sqlite3

In [2]:
db_connection = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
pd.read_sql(sql='PRAGMA table_info(test);', con=db_connection)

,cid,name,type,notnull,dflt_value,pk


In [4]:
pd.read_sql(sql='SELECT * FROM test LIMIT 10;', con=db_connection, index_col='index')

DatabaseError: Execution failed on sql 'SELECT * FROM test LIMIT 10;': no such table: test

In [ ]:
pd.read_sql(sql='PRAGMA table_info(deadlines);', con=db_connection)

In [ ]:
pd.read_sql(sql='SELECT * FROM deadlines LIMIT 10;', con=db_connection, index_col='index')

In [ ]:
sql_query="""
SELECT
    uid,
    MIN(diff)
FROM (
    SELECT
        test.uid,
        test.labname,
        CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT) AS diff
    FROM
        test
    INNER JOIN 
        deadlines
    ON
        test.labname != 'project1'
        AND test.labname = deadlines.labs
);
"""

df_min = pd.read_sql(sql=sql_query, con=db_connection)
df_min

In [ ]:
df_max = pd.read_sql("""SELECT uid, MAX(diff) FROM
                        (SELECT test.uid, test.labname,
                        CAST((julianday(test.first_commit_ts)
                        - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT) AS diff 
                        FROM test INNER JOIN  deadlines ON test.labname != 'project1' 
                        AND test.labname = deadlines.labs);""",
                        con=db_connection)
df_max

In [ ]:
df_avg = pd.read_sql("""SELECT AVG(diff) FROM
             (SELECT test.uid,
             test.labname,
             CAST((julianday(test.first_commit_ts) - 
             julianday(deadlines.deadlines, 'unixepoch'))
             * 24 AS INT) AS diff
             FROM test
             INNER JOIN  deadlines
             ON test.labname != 'project1'
             AND test.labname = deadlines.labs);""",
             con=db_connection)
df_avg

In [ ]:
views_diff = pd.read_sql("""SELECT uid, AVG(diff) AS avg_diff, COUNT(uid) as pageviews
             FROM ( SELECT test.uid, CAST((julianday(test.first_commit_ts) -
             julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT) AS diff
             FROM test INNER JOIN  deadlines ON test.labname != 'project1'
             AND test.labname = deadlines.labs) GROUP BY uid""", con=db_connection)
views_diff

In [ ]:
views_diff.corr()

In [ ]:
db_connection.close()